In [2]:
!pip3 install pdfkit --user

  Using cached https://files.pythonhosted.org/packages/64/1b/26c080096dd93936dccfd32c682bed3d5630a84aae9d493ff68afb2ae0fb/pdfkit-1.0.0-py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# for ubuntu or debian
# sudo apt-get install wkhtmltopdf

# for macOS
# brew install homebrew/cask/wkhtmltopdf

# docs
# https://wkhtmltopdf.org/usage/wkhtmltopdf.txt

In [1]:
import pdfkit, requests
from bs4 import BeautifulSoup

In [2]:
def get_children_url(_d):
    urls = []
    for _path in _d:
        if _path.get('path'):
            urls.append(_path['path'])
            continue

        if _path.get('children'):
            urls.extend(get_children_url(_path['children']))
            continue
    return urls

In [3]:
def get_urls(product, version, lang):
    if product == 'broker':
        json_url = f'https://raw.githubusercontent.com/emqx/emqx-docs/release-{version}/directory.json'
    elif product == 'enterprise':
        json_url = f'https://raw.githubusercontent.com/emqx/emqx-docs/release-{version}/directory_ee.json'
    else:
        print('error product', product)
        return None
    
    json_data = requests.get(json_url).json()
    json_data = json_data[lang]

    all_path = get_children_url(json_data)
    all_urls = []
    for _path in all_path:
        if lang == 'cn':
            lang = 'zh'
        if _path == './':
            all_urls.append(f'https://docs.emqx.com/{lang}/enterprise/v{version}/')
        else:
            all_urls.append(f'https://docs.emqx.com/{lang}/enterprise/v{version}/{_path}.html')

    return all_urls

In [54]:
url = get_urls('enterprise', '4.4', 'cn')
options = {
    'print-media-type': None,
    'user-style-sheet': 'my.css',
    'dump-outline': 'toc.xml',
    'enable-local-file-access': None,
    'javascript-delay': 60000,
    'header-center': 'EMQX Enterprise V4.4 Docs',
    'header-font-size': 10,
    'header-spacing': 5,
    'footer-center': '[page] / [topage]',
    'footer-font-size': 8,
    'page-offset': -1
#     'enable-internal-links': None,
#     'keep-relative-links': None
#     'dump-default-toc-xsl': None
}
toc = { 
    'xsl-style-sheet': 'toc.xsl'
}
cover = 'https://docs.emqx.com/404.html'

pdfkit.from_url(url, 'out.pdf', options=options, verbose=True, toc=toc, cover=cover, cover_first=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



True

In [ ]:
url = get_urls('enterprise', '4.4', 'cn')

In [72]:
# test command
r = pdfkit.PDFKit(url, 'url', options=options, verbose=True, toc=toc)
print(' '.join(r.command()))

/usr/local/bin/wkhtmltopdf --print-media-type --user-style-sheet my.css --dump-outline toc.xml --enable-local-file-access --javascript-delay 500 --header-center EMQX Enterprise V4.4 Docs --header-font-size 10 --header-spacing 5 --footer-center [page] / [topage] --footer-font-size 8 --page-offset -1 toc --xsl-style-sheet toc.xsl https://docs.emqx.com/zh/enterprise/v4.4/ https://docs.emqx.com/zh/enterprise/v4.4/getting-started/install-ee.html https://docs.emqx.com/zh/enterprise/v4.4/getting-started/start.html https://docs.emqx.com/zh/enterprise/v4.4/faq/error.html https://docs.emqx.com/zh/enterprise/v4.4/bridge/bridge.html https://docs.emqx.com/zh/enterprise/v4.4/rule/rule-engine.html https://docs.emqx.com/zh/enterprise/v4.4/getting-started/log.html -


In [20]:
with open('my.css', 'w') as f:
    f.write('''
  .navbar,
  .right-column,
  .main-footer-box {
    display: none;
  }
  .main-box {
    padding-top: 0;
  }
  #page-feedback {
    display: none;
  }
  
''')

In [23]:
s = '''
<?xml version="1.0" encoding="UTF-8"?>
<xsl:stylesheet version="2.0"
                xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
                xmlns:outline="http://wkhtmltopdf.org/outline"
                xmlns="http://www.w3.org/1999/xhtml">
  <xsl:output doctype-public="-//W3C//DTD XHTML 1.0 Strict//EN"
              doctype-system="http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"
              indent="yes" />
  <xsl:template match="outline:outline">
    <html>
      <head>
        <title>Table of Contents</title>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
        <style>
          h1 {
            text-align: center;
            font-size: 20px;
            font-family: arial;
          }
          div {border-bottom: 1px dashed rgb(200,200,200);}
          span {float: right;}
          li {list-style: none;}
          ul {
            font-size: 20px;
            font-family: arial;
          }
          ul ul {font-size: 80%; }
          ul {padding-left: 0em;}
          ul ul {padding-left: 1em;}
          a {text-decoration:none; color: black;}
        </style>
      </head>
      <body>
        <h1>Table of Contents</h1>
        <ul><xsl:apply-templates select="outline:item/outline:item"/></ul>
      </body>
    </html>
  </xsl:template>
  <xsl:template match="outline:item">
    <li>
      <xsl:if test="@title!=''">
        <div>
          <a>
            <xsl:if test="@link">
              <xsl:attribute name="href"><xsl:value-of select="@link"/></xsl:attribute>
            </xsl:if>
            <xsl:if test="@backLink">
              <xsl:attribute name="name"><xsl:value-of select="@backLink"/></xsl:attribute>
            </xsl:if>
            <xsl:value-of select="@title" /> 
          </a>
          <span> <xsl:value-of select="@page" /> </span>
        </div>
      </xsl:if>
      <ul>
        <xsl:comment>added to prevent self-closing tags in QtXmlPatterns</xsl:comment>
        <xsl:apply-templates select="outline:item"/>
      </ul>
    </li>
  </xsl:template>
</xsl:stylesheet>
'''

In [24]:
with open('toc.xsl', 'w') as f:
    f.write(s)